## Q-Learning Reinforcement Learning (Frozen Lake)

### Dependencies
```{bash}
pip install "gym[toy_text]" numpy
```

In [ ]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

slippery = True

env = gym.make("FrozenLake-v1", is_slippery=slippery)

action_space_size = env.action_space.n # Q-Table columns
state_space_size = env.observation_space.n # Q-Table rows

q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

In [ ]:
num_episodes = 10000 # games played
max_steps_per_episode = 100 # max step so we don't get stuck

learning_rate = 0.1
discount_rate = 0.99

# trade off between exploration and exploitation
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

rewards_all_episodes = []

for episode in range(num_episodes):
    state, _ = env.reset()

    rewards_current_episode = 0

    for step in range(max_steps_per_episode):
        # Exploration/Exploitation trade off (choose action)
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state]) # exploit
        else:
            action = env.action_space.sample() # explore

        new_state, reward, terminated, truncated, _ = env.step(action)

        # Q-Table update formula
        q_table[state][action] = q_table[state][action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state]))

        state = new_state
        rewards_current_episode += reward

        if terminated or truncated:
            break

    # exploration rate decay
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

    rewards_all_episodes.append(rewards_current_episode)

In [ ]:
# print average reward per 1000 episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("Average reward per thousand episodes")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

In [ ]:
print("Q-Table")
print(q_table)

In [ ]:
# play 3 episodes with trained Q-Table
env = gym.make("FrozenLake-v1", is_slippery=slippery, render_mode="human")

for episode in range(3):
    state, _ = env.reset()
    print("Episode ", episode + 1, "\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):
        clear_output(wait=True)

        action = np.argmax(q_table[state,:])
        new_state, reward, terminated, truncated, _ = env.step(action)

        if terminated or truncated:
            clear_output(wait=True)
            if reward == 1:
                print("You reached the goal!")
                time.sleep(1)
            else:
                print("You fell through a hole!")
                time.sleep(1)
            clear_output(wait=True)
            break

        state = new_state

env.close()

## Quellen / weiterführende Links
- [Gymnasium - Frozen Lake Environment](https://gymnasium.farama.org/environments/toy_text/frozen_lake/)
- [deeplizard - Reinforcement Learning Series](https://www.youtube.com/watch?v=nyjbcRQ-uQ8&list=PLZbbT5o_s2xoWNVdDudn51XM8lOuZ_Njv)